# Obliczenie dokładności na całych obrazach

### Wczytywanie koordynatów rzeczywistych i przewidzianych dla wszystkich obrazów

In [1]:
from validating_all_images import fddb_data

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
coords = fddb_data("c:/Dev/machine_learning_2/data/", show_images=False, verbose=False)

In [4]:
import numpy as np
coords = np.array(coords, dtype=object)

Pierwsza wartość to liczba wierszy, czyli liczba sprawdzonych obrzków.
Druga wartość to liczba kolumn. Pierwsza kolumna to nazwa pliku, druga kolumna to rzeczywiste bounding_boxes wczytane z plików, trzecia kolumna to bounding_boxes wraz z wartościami decyzji z detekcji na naszym modelu RealBoost.

In [5]:
coords.shape

(8, 3)

Pierwsza kolumna - nazwa pliku. Przyda się do późniejszego rysowania prostokątów.

In [6]:
coords[0][0]

'c:/Dev/machine_learning_2/data/2002/07/22/big/img_43.jpg'

Druga kolumna - rzeczywiste bounding boxes. W formacie:

```
np.array([j0, k0, j0 + w - 1, k0 + w - 1])
```

Aby wyrysować taki bounding box będziemy używać:

```
p1 = (k0, j0)
p2 = (k0 + w - 1, j0 + w - 1)
cv2.rectangle(i0, p1, p2, (0, 0, 255), 1)
cv2.imshow("FDDB", i0)
```

In [7]:
coords[0][1]

[array([ 53, 185, 155, 287])]

Trzecia kolumna - wykryte bounding boxes. Do rysowania tych bounding boxów użyjemy funkcji `draw_bounding_boxes`.

```
i0 = draw_bounding_boxes(i0, detections, color=(54, 193, 56), thickness=1)
cv2.imshow("FDDB", i0)
cv2.waitKey(0)
```

In [8]:
coords[0][2]

[(277, 115, 64, 3.052386965496715),
 (183, 45, 92, 4.027927659992461),
 (183, 54, 92, 4.0595346839655795),
 (192, 63, 92, 4.1576029966860375),
 (169, 45, 111, 4.501656264642891),
 (180, 45, 111, 5.635670238094397),
 (169, 56, 111, 4.195243147770823),
 (180, 56, 111, 4.271654083872954)]

### Uśrednianie wykrytych prostokątów

#### Progowanie

Wykryte prostokąty należy najpierw sprogować, czyli przefiltrować po danym progu. Wszystkie wykryte prostokąty to takie z wartością funkcji decyzyjnej $\gt 0$. Czemu tak? Żeby wygenerować wszystkie dane i móc sterować progiem decyzyjnym już po przeanalizowaniu wszystkich obrazków.

In [28]:
def reduce_by_threshold(image_row, **kwargs):
    threshold = kwargs['threshold']
    detections = np.array(image_row[2])
    if detections.ndim == 1:
        return (image_row[0], image_row[1], detections)
    
    detections_reduced = detections[np.where(detections[:, 3] > threshold)]
    return (image_row[0], image_row[1], detections_reduced)

coords_filtered = np.apply_along_axis(reduce_by_threshold, 1, coords, threshold=5)

#### Uśrednianie

Uśrednianie prostokątów już po filtracji wybranym progiem.

In [30]:
from haar_students_new import non_max_suppression

def bounding_boxes_reduction(image_row):
    detections = image_row[2]
    detections_reduced = non_max_suppression(detections, treshold=0.1)
    return (image_row[0], image_row[1], detections_reduced)

coords_suppresion = np.apply_along_axis(bounding_boxes_reduction, 1, coords_filtered)
coords_suppresion

array([['c:/Dev/machine_learning_2/data/2002/07/22/big/img_43.jpg',
        list([array([ 53, 185, 155, 287])]),
        list([(180, 45, 111, 5.635670238094397)])],
       ['c:/Dev/machine_learning_2/data/2002/07/26/big/img_634.jpg',
        list([array([ 56, 116, 142, 202]), array([ 55, 260, 141, 346])]),
        list([(255, 55, 92, 5.536743062695408)])],
       ['c:/Dev/machine_learning_2/data/2002/07/19/big/img_843.jpg',
        list([array([ 61,  55, 169, 163]), array([ 26, 223,  90, 287]), array([ 54, 326, 160, 432])]),
        list([(59, 60, 111, 6.018710650164764)])],
       ['c:/Dev/machine_learning_2/data/2002/08/26/big/img_58.jpg',
        list([array([ 25,  96, 105, 176])]), list([])],
       ['c:/Dev/machine_learning_2/data/2002/08/31/big/img_18008.jpg',
        list([array([ 80, 160, 163, 243]), array([ 93, 253, 173, 333]), array([135, 358, 200, 423])]),
        list([])],
       ['c:/Dev/machine_learning_2/data/2002/08/22/big/img_249.jpg',
        list([array([ 32,  64, 1

### Obliczanie dokładności na danych

Dla każdego uśrednionego prostokąta sprawdzamy czy się pokrywa z innym zgodnie z wymaganiem `iou > 0.5`. 